In [1]:
import numpy as np
import pandas as pd
import tqdm

import catboost as cat
from catboost import CatBoostClassifier

## Загрузим данные

In [2]:
%%time
edges = pd.read_csv('./edges.csv')
ids = pd.read_csv('./ids.csv')
vertices = pd.read_csv('./vertices.csv')

CPU times: user 3.38 s, sys: 329 ms, total: 3.71 s
Wall time: 3.71 s


In [3]:
vertices['main_okved'] = vertices['main_okved'].astype(str)

In [4]:
np.random.seed(7777)

## Обучим модель

In [5]:
result = pd.DataFrame(columns=['id_1', 'id_2'])

In [6]:
# для каждой вершины из ids с помощью catboost найдем 1000 самых вероятных ребер
for i in tqdm.tqdm(ids.id):
    # соберем датасет из всех возможных вершин
    # вершины имеющие в исходных данных ребро с i обозначим 1, остальные 0
    # учтем то, что вершина i может быть как среди id_1, так и среди id_2
    df1 = edges[edges['id_1'] == i].reset_index()
    df2 = edges[edges['id_2'] == i].reset_index()

    df = df1[['id_2', 'id_1']].rename(columns={'id_1':'id_2', 'id_2':'id_1'}).append(df2[['id_1', 'id_2']])
    df['target'] = 1
    
    df = vertices.set_index('id').join(df.set_index('id_1')['target']).fillna(0)
    
    
    X = df[['main_okved', 'region_code', 'company_type']]
    y = df['target']
    
    model = CatBoostClassifier(iterations=100, verbose=False)
    cat_features = [0,1,2] # все признаки категориальные
    
    model.fit(X, y, cat_features)

    preds = model.predict_proba(X)[:,1]

    df['preds'] = preds
    df['id_2'] = i
    
    # возьмем первую 1000 предсказанных ребер, исключив те, про которые мы уже знали
    res = df[df['target'] != 1].sort_values(by='preds', ascending=False).iloc[:1000].reset_index()[['id', 'id_2']]
    res.columns = ['id_1', 'id_2']
    
    result = result.append(res, ignore_index=True, sort=False)

 28%|██▊       | 28/100 [26:00<1:05:35, 54.66s/it]

Training has stopped (degenerate solution on iteration 93, probably too small l2-regularization, try to increase it)


100%|██████████| 100/100 [1:32:34<00:00, 54.28s/it]


## Результат готов к отправке

In [7]:
result.to_csv('submission.csv', index = False)